In [1]:
import os

project_path = r"C:\Users\user\anaconda3\envs\Chicken-disease-Classification-Project"

os.chdir(project_path)
print(f"Moved to: {os.getcwd()}")


Moved to: C:\Users\user\anaconda3\envs\Chicken-disease-Classification-Project


In [2]:
%pwd

'C:\\Users\\user\\anaconda3\\envs\\Chicken-disease-Classification-Project'

In [ ]:
from dataclasses import dataclass
from pathlib import Path
from typing import List
#entity
@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path                          # Not in YAML – likely set in code
    base_model_dir: Path                    # Not in YAML – likely set in code
    updated_base_model_dir: Path            # Not in YAML – likely set in code
    params_image_size: List[int]            # From YAML → MODEL_PARAMS.IMAGE_SIZE
    params_learning_rate: float             # From YAML → MODEL_PARAMS.LEARNING_RATE
    params_include_top: bool                # From YAML → MODEL_PARAMS.INCLUDE_TOP
    params_weights: str                     # From YAML → MODEL_PARAMS.WEIGHTS
    params_classes: int 

In [4]:
from cnnclassifier.constants import *
from cnnclassifier.utils.common import  read_yaml, create_directories # Assumed utility file

In [ ]:
#configuration manager
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAM_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config['artifacts_root']])  # ✅ This is correct

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config['prepare_base_model']

        create_directories([config['root_dir']])  # ✅ Treat as dictionary

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config['root_dir']),
            base_model_dir=Path(config['base_model_dir']),
            updated_base_model_dir=Path(config['updated_base_model_dir']),
            params_image_size=self.params['MODEL_PARAMS']['IMAGE_SIZE'],
            params_learning_rate=self.params['MODEL_PARAMS']['LEARNING_RATE'],
            params_include_top=self.params['MODEL_PARAMS']['INCLUDE_TOP'],
            params_weights=self.params['MODEL_PARAMS']['WEIGHTS'],
            params_classes=self.params['MODEL_PARAMS']['CLASSES']
        )

        return prepare_base_model_config

In [ ]:
# component
import os
import urllib.request as request
from zipfile import ZipFile, BadZipFile
import tensorflow as tf
from pathlib import Path

class PrepareBaseModel:
    def __init__(self, config):
        self.config = config
        self.model = None
        self.full_model = None

    def get_base_model(self):
        self.model = tf.keras.applications.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top,
            classes=self.config.params_classes
        )

        self.save_model(
            path=self.config.base_model_dir,
            model=self.model
        )

    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False
        elif freeze_till is not None and freeze_till > 0:
            for layer in model.layers[:freeze_till]:
                layer.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(classes, activation='softmax')(flatten_in)

        full_model = tf.keras.models.Model(inputs=model.input, outputs=prediction)
        full_model.compile(
            loss='categorical_crossentropy',
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            metrics=['accuracy']
        )
        full_model.summary()
        return full_model

    def update_base_model(self):
        self.full_model = PrepareBaseModel._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(
            path=self.config.updated_base_model_dir,
            model=self.full_model
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


In [10]:
import logging

logging.basicConfig(level=logging.INFO)

try:
    logging.info("Starting prepare base model pipeline...")
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    
    logging.info("Getting base model...")
    prepare_base_model.get_base_model()
    
    logging.info("Updating base model...")
    prepare_base_model.update_base_model()
    
    logging.info("Pipeline completed successfully.")
except Exception as e:
    logging.error(f"Pipeline failed due to: {e}")
    raise


[2025-05-26 11:19:23,786: INFO: 3191992802: Starting prepare base model pipeline...]
[2025-05-26 11:19:23,808: INFO: common: YAML file: config\config.yaml loaded successfully]
[2025-05-26 11:19:23,826: INFO: common: YAML file: params.yaml loaded successfully]
[2025-05-26 11:19:23,832: INFO: common: Created directory at: artifacts]
[2025-05-26 11:19:23,842: INFO: common: Created directory at: artifacts/prepare_base_model]
[2025-05-26 11:19:23,847: INFO: 3191992802: Getting base model...]
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 22s 0us/step
[2025-05-26 11:19:50,916: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
[2025-05-26 11:19:51,281: INFO: 3191992802: Updating base model...]


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │        50,178 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,764,866 (56.32 MB)

 Trainable params: 50,178 (196.01 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

[2025-05-26 11:19:51,597: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
[2025-05-26 11:19:52,045: INFO: 3191992802: Pipeline completed successfully.]
